In [2]:
# import library
import tensorflow as tf
import numpy as np
import sys, os,cv2
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle
import matplotlib.animation as animation
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs,make_moons,make_swiss_roll
from IPython.display  import IFrame

plt.style.use('seaborn')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sess = tf.InteractiveSession()

In [26]:
# read all of the data
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
test_images  = read_all_images("../../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
test_labels  = read_labels    ("../../../DataSet/STL10/stl10_binary/test_y.bin")

label_encoder= OneHotEncoder(sparse=False,categories='auto')
test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))
print(test_images.shape)
print(test_labels.shape)

(8000, 96, 96, 3)
(8000, 10)


In [27]:
# shuffle and choose some 
test_images = test_images[np.random.permutation(range(8000))][:20,:,:,:]
print(test_images.shape)

(20, 96, 96, 3)


In [32]:
# show s u v
test_images_flat = np.reshape(test_images,(20,96*96,3))
s,U,V   = tf.svd(test_images_flat)
smin    = tf.reduce_min(s,1,keepdims=True)
smax    = tf.reduce_max(s,1,keepdims=True)
scaleds = (s - smin)/(smax - sm)
print(s.shape,U.shape,V.shape)
print(s.eval())
print(smin.eval())

(20, 3) (20, 9216, 3) (20, 3, 3)
[[ 85.23419885  10.09639759   5.03899525]
 [ 99.76483989   4.38067074   0.68340723]
 [ 85.52585557   4.88880737   2.04994964]
 [104.65363803   2.92912635   2.15039383]
 [ 91.73792115   3.3337852    1.48640598]
 [ 96.09514678   5.26032663   3.61897189]
 [ 66.73022029   6.24293727   2.66518949]
 [ 76.41944925  14.94471729   1.46516197]
 [116.20832409   8.75029012   1.80756128]
 [ 67.80445162   7.92808671   4.47123383]
 [ 81.15044884   6.53810427   1.66171993]
 [ 85.68737958   8.31203833   2.73003311]
 [ 70.26765363   3.37747005   1.58940633]
 [ 88.13917902   8.66685819   2.21592273]
 [ 72.24368186   7.11945119   2.0016453 ]
 [ 77.8107459    9.1888384    1.98184751]
 [108.45456837   5.25632701   1.04413289]
 [ 99.924446     3.41020751   1.00203039]
 [ 75.07205628   4.11915938   2.8707447 ]
 [ 74.34558614  11.07276207   1.79037843]]
[[5.03899525]
 [0.68340723]
 [2.04994964]
 [2.15039383]
 [1.48640598]
 [3.61897189]
 [2.66518949]
 [1.46516197]
 [1.80756128]
